# Advanced Seismic Analytics: Real-time Event Detection and Analysis

This notebook demonstrates the advanced analytics capabilities of the seismic classifier system, including:
- Real-time event detection using STA/LTA and deep learning
- Magnitude estimation with confidence intervals
- Location determination with uncertainty quantification
- Statistical analysis and alert system implementation

The examples use synthetic data for demonstration, but the same techniques can be applied to real seismic data.

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import folium
from scipy import signal, stats
from obspy import Stream, Trace
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from tensorflow import keras
import asyncio
from aiohttp import ClientSession
from asyncio_throttle import Throttler

# Import our custom modules
from src.seismic_classifier.advanced_analytics import (
    RealTimeDetector,
    MagnitudeEstimator,
    LocationDeterminer,
    ConfidenceAnalyzer
)

# Set plot styles
plt.style.use('seaborn')
sns.set_theme(style="whitegrid")

# Configure notebook display
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# 1. Real-time Event Detection System

We'll start by implementing a real-time event detection system using the STA/LTA (Short-Term Average/Long-Term Average) algorithm combined with deep learning for validation. This approach provides robust detection capabilities while minimizing false positives.

In [ ]:
# Generate synthetic seismic data with known events
def generate_synthetic_event(
    duration=60,
    sampling_rate=100,
    event_time=30,
    magnitude=3.0,
    noise_level=0.1
):
    """Generate synthetic seismic waveform with an event."""
    t = np.linspace(0, duration, int(duration * sampling_rate))
    
    # Background noise
    noise = noise_level * np.random.randn(len(t))
    
    # Event waveform (simplified P and S waves)
    p_wave = magnitude * np.exp(-(t - event_time)**2) * np.sin(2*np.pi*5*(t - event_time))
    s_wave = 1.5 * magnitude * np.exp(-(t - (event_time + 2))**2) * np.sin(2*np.pi*2*(t - (event_time + 2)))
    
    # Combine components
    waveform = p_wave + s_wave + noise
    
    return t, waveform

# Generate example data
duration = 120  # 2 minutes
sampling_rate = 100  # 100 Hz
t, waveform = generate_synthetic_event(
    duration=duration,
    sampling_rate=sampling_rate,
    event_time=60,
    magnitude=3.5
)

# Plot the synthetic waveform
plt.figure(figsize=(15, 5))
plt.plot(t, waveform)
plt.title('Synthetic Seismic Waveform')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.grid(True)
plt.show()

In [ ]:
# Initialize real-time detector
detector = RealTimeDetector(
    sta_length=1.0,
    lta_length=20.0,
    sta_lta_threshold=3.0,
    sampling_rate=sampling_rate
)

# Create ObsPy Stream for processing
trace = Trace(data=waveform)
trace.stats.sampling_rate = sampling_rate
stream = Stream([trace])

# Process stream and detect events
async def detect_events():
    events = await detector.process_stream(stream)
    return events

# Run event detection
events = asyncio.run(detect_events())

# Plot waveform with detected events
plt.figure(figsize=(15, 5))
plt.plot(t, waveform, 'b-', label='Waveform', alpha=0.7)

# Mark detected events
for event in events:
    start_idx = int(event['start_time'] * sampling_rate)
    end_idx = int(event['end_time'] * sampling_rate)
    plt.axvspan(t[start_idx], t[end_idx], color='red', alpha=0.3)
    plt.axvline(t[start_idx], color='red', linestyle='--', alpha=0.5)
    
plt.title('Seismic Waveform with Detected Events')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.legend()
plt.grid(True)
plt.show()

# Print event details
for i, event in enumerate(events):
    print(f"\nEvent {i+1}:")
    print(f"Start time: {event['start_time']:.2f}s")
    print(f"End time: {event['end_time']:.2f}s")
    print(f"Max STA/LTA: {event['max_sta_lta']:.2f}")
    print(f"Confidence: {event.get('confidence', 'N/A')}"))

# 2. Magnitude Estimation

Now that we've detected events in our waveform, let's estimate their magnitudes using both traditional amplitude-based methods and machine learning approaches. We'll also calculate confidence intervals for our estimates.

In [ ]:
# Initialize magnitude estimator
magnitude_estimator = MagnitudeEstimator(
    sampling_rate=sampling_rate,
    window_size=60.0
)

# Extract event waveforms and estimate magnitudes
event_waveforms = []
for event in events:
    start_idx = int(event['start_time'] * sampling_rate)
    end_idx = int(event['end_time'] * sampling_rate)
    event_waveforms.append(waveform[start_idx:end_idx])

# Estimate magnitudes for all events
magnitude_results = magnitude_estimator.batch_estimate(event_waveforms)

# Create a results dataframe
results_df = pd.DataFrame([
    {
        'Event': i+1,
        'Start Time': event['start_time'],
        'End Time': event['end_time'],
        'Duration': event['end_time'] - event['start_time'],
        'Magnitude': mag_result['magnitude'],
        'Uncertainty': mag_result['uncertainty'],
        'Confidence': mag_result['confidence']
    }
    for i, (event, mag_result) in enumerate(zip(events, magnitude_results))
])

# Display results
print("\nMagnitude Estimation Results:")
print(results_df.to_string(index=False))

# Plot magnitudes with error bars
plt.figure(figsize=(10, 6))
plt.errorbar(
    results_df['Event'],
    results_df['Magnitude'],
    yerr=results_df['Uncertainty'],
    fmt='o',
    capsize=5,
    capthick=1,
    elinewidth=1,
    markersize=8
)
plt.title('Event Magnitudes with Uncertainty')
plt.xlabel('Event Number')
plt.ylabel('Estimated Magnitude')
plt.grid(True)
plt.show()

# Create interactive plot with Plotly
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=results_df['Event'],
    y=results_df['Magnitude'],
    error_y=dict(
        type='data',
        array=results_df['Uncertainty'],
        visible=True
    ),
    mode='markers',
    name='Magnitude',
    hovertemplate='Event: %{x}<br>Magnitude: %{y:.2f}<br>Uncertainty: %{error_y.array:.2f}'
))

fig.update_layout(
    title='Interactive Event Magnitude Plot',
    xaxis_title='Event Number',
    yaxis_title='Estimated Magnitude',
    showlegend=True
)

fig.show()

# 3. Event Location Determination

In this section, we'll demonstrate how to determine the location of seismic events using arrival times from multiple stations. We'll use both traditional triangulation methods and ML-enhanced approaches with uncertainty quantification.

In [ ]:
# Define station network
station_coords = {
    'STA1': (0.0, 0.0, 0.0),      # Reference station
    'STA2': (50.0, 0.0, 0.0),     # 50km East
    'STA3': (0.0, 50.0, 0.0),     # 50km North
    'STA4': (-30.0, -30.0, 0.0)   # Southwest
}

# Initialize location determiner
location_determiner = LocationDeterminer(station_coords=station_coords)

# Generate synthetic arrival times for a known event location
true_location = {
    'latitude': 25.0,
    'longitude': 25.0,
    'depth': 10.0
}

# Create synthetic arrival times with some noise
def generate_arrival_times(location, stations, velocity_model={'P': 6.0, 'S': 3.5}):
    arrival_times = {}
    reference_station = list(stations.keys())[0]
    ref_dist = np.sqrt(
        sum((x - y) ** 2 for x, y in zip(
            stations[reference_station],
            (location['latitude'], location['longitude'], location['depth'])
        ))
    )
    
    for station, coords in stations.items():
        # Calculate distance
        distance = np.sqrt(
            sum((x - y) ** 2 for x, y in zip(
                coords,
                (location['latitude'], location['longitude'], location['depth'])
            ))
        )
        
        # Add random timing errors
        p_error = np.random.normal(0, 0.1)  # 0.1s standard deviation
        s_error = np.random.normal(0, 0.2)  # 0.2s standard deviation
        
        # Calculate arrival times
        arrival_times[station] = {
            'P': distance / velocity_model['P'] + p_error,
            'S': distance / velocity_model['S'] + s_error
        }
    
    return arrival_times

# Generate arrival times
arrival_times = generate_arrival_times(true_location, station_coords)

# Locate event
estimated_location = location_determiner.locate_event(arrival_times)

# Create interactive map
center_lat = np.mean([coords[0] for coords in station_coords.values()])
center_lon = np.mean([coords[1] for coords in station_coords.values()])

m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=8,
    tiles='cartodbpositron'
)

# Add stations
for name, coords in station_coords.items():
    folium.CircleMarker(
        location=[coords[0], coords[1]],
        radius=8,
        color='blue',
        fill=True,
        popup=name
    ).add_to(m)

# Add true location
folium.CircleMarker(
    location=[true_location['latitude'], true_location['longitude']],
    radius=10,
    color='green',
    fill=True,
    popup='True Location'
).add_to(m)

# Add estimated location with uncertainty
folium.Circle(
    location=[estimated_location['latitude'], estimated_location['longitude']],
    radius=estimated_location['uncertainty'] * 1000,  # Convert km to meters
    color='red',
    fill=True,
    popup=f"Estimated Location<br>Confidence: {estimated_location['confidence']:.2f}"
).add_to(m)

m

# Print results comparison
print("\nLocation Estimation Results:")
print("\nTrue Location:")
print(f"Latitude: {true_location['latitude']:.2f}°")
print(f"Longitude: {true_location['longitude']:.2f}°")
print(f"Depth: {true_location['depth']:.2f} km")

print("\nEstimated Location:")
print(f"Latitude: {estimated_location['latitude']:.2f}° ± {estimated_location['uncertainty']:.2f}")
print(f"Longitude: {estimated_location['longitude']:.2f}° ± {estimated_location['uncertainty']:.2f}")
print(f"Depth: {estimated_location['depth']:.2f} km ± {estimated_location['uncertainty']:.2f}")
print(f"Confidence: {estimated_location['confidence']:.2%}")
print(f"Stations used: {', '.join(estimated_location['stations_used'])}")

# 4. Statistical Analysis and Confidence Intervals

Finally, let's perform a comprehensive statistical analysis of our detection and estimation results, calculating confidence intervals and performing uncertainty quantification for all parameters.

In [ ]:
# Initialize confidence analyzer
confidence_analyzer = ConfidenceAnalyzer(
    bootstrap_iterations=1000,
    confidence_level=0.95
)

# Analyze detection confidence
detection_results = []
for event, waveform in zip(events, event_waveforms):
    detection_params = {
        'sta_lta': event['max_sta_lta'],
        'snr': np.max(np.abs(waveform)) / np.std(waveform)
    }
    
    result = confidence_analyzer.analyze_detection_confidence(
        waveform,
        detection_params
    )
    detection_results.append(result)

# Analyze magnitude confidence
magnitude_confidence = [
    confidence_analyzer.analyze_magnitude_confidence(
        result['magnitude'],
        waveform,
        noise_level=0.1
    )
    for result, waveform in zip(magnitude_results, event_waveforms)
]

# Analyze location confidence
location_confidence = confidence_analyzer.analyze_location_confidence(
    estimated_location,
    arrival_times,
    {'P': 6.0, 'S': 3.5}
)

# Create comprehensive results visualization
def plot_confidence_results(confidence_data, title, param_name):
    fig = go.Figure()
    
    # Add confidence intervals
    intervals = confidence_data['confidence_intervals']
    for param, (lower, upper) in intervals.items():
        fig.add_trace(go.Scatter(
            x=[lower, upper],
            y=[param, param],
            mode='lines',
            name=param,
            line=dict(width=10, color='rgba(68, 166, 198, 0.6)'),
            hovertemplate=f'Parameter: {param}<br>Range: {lower:.2f} - {upper:.2f}'
        ))
        
        # Add point estimate
        mean = (lower + upper) / 2
        fig.add_trace(go.Scatter(
            x=[mean],
            y=[param],
            mode='markers',
            name=f'{param} (estimate)',
            marker=dict(size=10, color='blue'),
            hovertemplate=f'Estimate: {mean:.2f}'
        ))
    
    fig.update_layout(
        title=title,
        xaxis_title=param_name,
        yaxis_title='Parameter',
        showlegend=False,
        height=400
    )
    
    return fig

# Plot confidence results for detection
detection_fig = plot_confidence_results(
    detection_results[0],
    'Detection Confidence Intervals',
    'Value'
)
detection_fig.show()

# Plot confidence results for magnitude
magnitude_fig = go.Figure()

for i, conf in enumerate(magnitude_confidence):
    magnitude_fig.add_trace(go.Scatter(
        x=[i+1],
        y=[conf['magnitude']],
        error_y=dict(
            type='data',
            array=[conf['uncertainty']],
            visible=True
        ),
        mode='markers',
        name=f'Event {i+1}',
        hovertemplate=(
            f'Event {i+1}<br>'
            f'Magnitude: {conf["magnitude"]:.2f}<br>'
            f'Uncertainty: ±{conf["uncertainty"]:.2f}<br>'
            f'Confidence: {conf["confidence"]:.2%}'
        )
    ))

magnitude_fig.update_layout(
    title='Magnitude Estimates with Confidence Intervals',
    xaxis_title='Event Number',
    yaxis_title='Magnitude',
    showlegend=True
)

magnitude_fig.show()

# Plot location confidence
location_fig = plot_confidence_results(
    location_confidence,
    'Location Confidence Intervals',
    'Distance (km)'
)
location_fig.show()

# Print summary statistics
print("\nConfidence Analysis Summary:")
print("\nDetection Confidence:")
for i, result in enumerate(detection_results):
    print(f"\nEvent {i+1}:")
    print(f"Overall confidence: {result['overall_confidence']:.2%}")
    print("Confidence intervals:")
    for param, (lower, upper) in result['confidence_intervals'].items():
        print(f"  {param}: {lower:.2f} - {upper:.2f}")

print("\nMagnitude Confidence:")
for i, result in enumerate(magnitude_confidence):
    print(f"\nEvent {i+1}:")
    print(f"Magnitude: {result['magnitude']:.2f} ± {result['uncertainty']:.2f}")
    print(f"Confidence: {result['confidence']:.2%}")
    if result['snr'] is not None:
        print(f"SNR: {result['snr']:.1f} dB")

print("\nLocation Confidence:")
print(f"Average uncertainty: {location_confidence['average_uncertainty']:.2f} km")
print(f"Maximum uncertainty: {location_confidence['max_uncertainty']:.2f} km")
print(f"Overall confidence: {location_confidence['confidence']:.2%}")
print("\nParameter uncertainties:")
for param, uncertainty in location_confidence['uncertainties'].items():
    print(f"  {param}: ±{uncertainty:.2f} km")

# 5. Alert System Implementation

Finally, let's implement a real-time alert system that can notify users of detected events based on configurable criteria. This system uses async/await for efficient handling of multiple notifications.

In [ ]:
# Define alert system class
class SeismicAlertSystem:
    def __init__(
        self,
        magnitude_threshold=3.0,
        confidence_threshold=0.8,
        throttle_rate=1.0  # Maximum alerts per second
    ):
        self.magnitude_threshold = magnitude_threshold
        self.confidence_threshold = confidence_threshold
        self.throttler = Throttler(rate_limit=throttle_rate)
        self.alerts = []
        
    async def process_event(self, event, magnitude_result, location_result):
        """Process a detected event and generate alerts if needed."""
        if (magnitude_result['magnitude'] >= self.magnitude_threshold and
            magnitude_result['confidence'] >= self.confidence_threshold):
            
            async with self.throttler:
                alert = {
                    'timestamp': pd.Timestamp.now(),
                    'magnitude': magnitude_result['magnitude'],
                    'confidence': magnitude_result['confidence'],
                    'location': {
                        'latitude': location_result['latitude'],
                        'longitude': location_result['longitude'],
                        'depth': location_result['depth']
                    },
                    'uncertainty': {
                        'magnitude': magnitude_result['uncertainty'],
                        'location': location_result['uncertainty']
                    }
                }
                
                self.alerts.append(alert)
                await self._send_alert(alert)
                
    async def _send_alert(self, alert):
        """Simulate sending an alert (would connect to actual notification service)."""
        print(f"\nALERT: Significant seismic event detected!")
        print(f"Time: {alert['timestamp']}")
        print(f"Magnitude: {alert['magnitude']:.1f} ± {alert['uncertainty']['magnitude']:.1f}")
        print(f"Location: {alert['location']['latitude']:.2f}°N, "
              f"{alert['location']['longitude']:.2f}°E")
        print(f"Depth: {alert['location']['depth']:.1f} km")
        print(f"Confidence: {alert['confidence']:.1%}")

# Initialize alert system
alert_system = SeismicAlertSystem(
    magnitude_threshold=3.0,
    confidence_threshold=0.8
)

# Process existing events through alert system
async def process_alerts():
    for event, mag_result, loc in zip(
        events,
        magnitude_results,
        [estimated_location] * len(events)  # Use same location for demo
    ):
        await alert_system.process_event(event, mag_result, loc)

# Run alert processing
asyncio.run(process_alerts())

# Create alert visualization
if alert_system.alerts:
    alerts_df = pd.DataFrame(alert_system.alerts)
    
    # Plot alert timeline
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=alerts_df['timestamp'],
        y=alerts_df['magnitude'],
        mode='markers+text',
        text=alerts_df.index,
        textposition='top center',
        marker=dict(
            size=alerts_df['confidence'] * 20,
            color=alerts_df['magnitude'],
            colorscale='Viridis',
            showscale=True,
            colorbar=dict(title='Magnitude')
        ),
        hovertemplate=(
            'Alert %{text}<br>'
            'Time: %{x}<br>'
            'Magnitude: %{y:.1f}<br>'
            'Confidence: %{marker.size:.0%}'
        )
    ))
    
    fig.update_layout(
        title='Seismic Event Alerts Timeline',
        xaxis_title='Time',
        yaxis_title='Magnitude',
        showlegend=False
    )
    
    fig.show()

# Conclusion

In this notebook, we've demonstrated the advanced analytics capabilities of our seismic classifier system, including:

1. Real-time event detection using combined STA/LTA and deep learning approaches
2. Magnitude estimation with confidence intervals and uncertainty quantification
3. Event location determination using arrival times and advanced triangulation
4. Statistical analysis and confidence bounds for all parameters
5. Real-time alert system implementation with rate limiting and notifications

These tools provide a comprehensive framework for analyzing seismic events with robust uncertainty quantification and real-time monitoring capabilities. The system can be extended with additional features and integrated with external notification services for production deployment.